<a href="https://colab.research.google.com/github/alessioborgi/DL_Project/blob/main/try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from datasets import load_dataset

In [3]:
# Load the MS MARCO (100K) Dataset
ms_marco = load_dataset("microsoft/ms_marco", "v1.1", split="train")


# Display a sample
print(ms_marco[0])

train_data = ms_marco.shuffle(seed=42).select(range(80000))
validation_data = ms_marco.shuffle(seed=42).select(range(8000))

README.md:   0%|          | 0.00/9.48k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/175M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

{'answers': ['Results-Based Accountability is a disciplined way of thinking and taking action that communities can use to improve the lives of children, youth, families, adults and the community as a whole.'], 'passages': {'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'passage_text': ["Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.", "The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank and banknote issuing authority, when the Reserve Bank Act 1959 removed the central banking functions from the Commonw

In [4]:
from sklearn.cluster import AgglomerativeClustering
from transformers import AutoModel, AutoTokenizer

# Load a pre-trained model for embeddings
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Generate embeddings for documents
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
for doc in train_data:
    print(doc)
    break

{'answers': ['Difficult to talk and breathe.'], 'passages': {'is_selected': [0, 0, 0, 0, 0, 1, 0], 'passage_text': ['A sore throat refers to pain, itchiness, or irritation of the throat. You may have difficulty swallowing food and liquids, and the pain may get worse when you try to swallow. A bacterial infection can also cause a sore throat. These types of infections include: 1  strep throat: inflammation of the throat caused by the Streptococcal bacteria. 2  diphtheria: infectious disease that causes throat inflammation.', 'Viruses cause most sore throats, but bacteria, such as streptococcus, which causes strep throat, may be the cause of a sore throat. An antibiotic can help get rid of the bacteria and provide relief. If your sore throat isn’t gone within a week, or if it gets worse and leads to a fever, you may need a prescription. ', 'Feel better fast. by Amanda MacMillan. A sore throat can be the first sign of a cold, a side effect of strained vocal cords, or an indication of some

In [8]:
for doc in train_data:
    print(doc.keys())
    break

dict_keys(['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'])


In [ ]:
# Create embeddings
embeddings = []
for i,doc in enumerate(train_data):
    print(i)
    embeddings.append(embed_text(doc['passages']['passage_text']))
#embeddings = [embed_text(doc['passages']['passage_text']) for doc in train_data]

# Apply clustering
clustering = AgglomerativeClustering(n_clusters=100)
labels = clustering.fit_predict(embeddings)

# Assign semantically structured identifiers
ssids = [f"Cluster_{label}" for label in labels]
train_data = [{'text': doc['passage_text'], 'docid': ssid} for doc, ssid in zip(train_data, ssids)]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [ ]:
from torch.utils.data import Dataset

class DSIInputs2TargetDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        doc_text = self.data[idx]['text']
        docid = self.data[idx]['docid']

        # Tokenize input (document text)
        source = self.tokenizer(
            doc_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Tokenize target (structured identifier)
        target = self.tokenizer(
            docid,
            max_length=10,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': source['input_ids'].squeeze(),
            'attention_mask': source['attention_mask'].squeeze(),
            'labels': target['input_ids'].squeeze()
        }


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

# Initialize model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Create the dataset
train_dataset = DSIInputs2TargetDataset(train_data, tokenizer)
eval_dataset = DSIInputs2TargetDataset(validation_data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./dsi_output",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(model, dataset):
    model.eval()
    hits_at_1 = 0
    total = len(dataset)

    for data in dataset:
        inputs = tokenizer(data['text'], return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model.generate(**inputs)
        predicted_docid = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if predicted_docid == data['docid']:
            hits_at_1 += 1

    accuracy = hits_at_1 / total
    print(f"Hits@1 Accuracy: {accuracy * 100:.2f}%")

# Run evaluation
evaluate(model, eval_dataset)
